In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/nba-games/players.csv
/kaggle/input/nba-games/teams.csv
/kaggle/input/nba-games/ranking.csv
/kaggle/input/nba-games/games_details.csv
/kaggle/input/nba-games/games.csv


LIST OF TO DOs:
1. Basic EDA on each dataset.
2. Merge some datasets and plot things. 
    Plot average points per team home and away 
    Plot wins vs shot %
    

# Importing packages.

For storage and manipulation of data we use pandas and numpy. For visualisations I am experimenting with plotly.

In [2]:
import pandas as pd
import numpy as np 
import plotly.express as px
from IPython.display import display

# Data Analysis

Firsly we'll load the data into dataframes and check it all has converted correctly.

Secondly we will plot the distribution of some interesting columns and relationships for each dataset. This will require some merging of datasets.

#### Games

In [3]:
games = pd.read_csv('/kaggle/input/nba-games/games.csv')

In [4]:
games.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,25.0,46.0,1610612759,117.0,0.478,0.815,0.321,23.0,44.0,1
1,2022-12-22,22200478,Final,1610612762,1610612764,2022,1610612762,120.0,0.488,0.952,...,16.0,40.0,1610612764,112.0,0.561,0.765,0.333,20.0,37.0,1
2,2022-12-21,22200466,Final,1610612739,1610612749,2022,1610612739,114.0,0.482,0.786,...,22.0,37.0,1610612749,106.0,0.470,0.682,0.433,20.0,46.0,1
3,2022-12-21,22200467,Final,1610612755,1610612765,2022,1610612755,113.0,0.441,0.909,...,27.0,49.0,1610612765,93.0,0.392,0.735,0.261,15.0,46.0,1
4,2022-12-21,22200468,Final,1610612737,1610612741,2022,1610612737,108.0,0.429,1.000,...,22.0,47.0,1610612741,110.0,0.500,0.773,0.292,20.0,47.0,0


In [5]:
games.shape

(26651, 21)

In [6]:
games.columns

Index(['GAME_DATE_EST', 'GAME_ID', 'GAME_STATUS_TEXT', 'HOME_TEAM_ID',
       'VISITOR_TEAM_ID', 'SEASON', 'TEAM_ID_home', 'PTS_home', 'FG_PCT_home',
       'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home', 'TEAM_ID_away',
       'PTS_away', 'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away',
       'REB_away', 'HOME_TEAM_WINS'],
      dtype='object')

Checking for any NaN values in the dataset

In [7]:
games.isna().sum()

GAME_DATE_EST        0
GAME_ID              0
GAME_STATUS_TEXT     0
HOME_TEAM_ID         0
VISITOR_TEAM_ID      0
SEASON               0
TEAM_ID_home         0
PTS_home            99
FG_PCT_home         99
FT_PCT_home         99
FG3_PCT_home        99
AST_home            99
REB_home            99
TEAM_ID_away         0
PTS_away            99
FG_PCT_away         99
FT_PCT_away         99
FG3_PCT_away        99
AST_away            99
REB_away            99
HOME_TEAM_WINS       0
dtype: int64

The na values will have to be dealt with.  

There are three primary ways to deal with this. Each NA value could be made zero, in this case that would not be the best option due to the way this would skew the data given that the columns with NA are pts, shot %, assists and rebounds.   
 
The other two ways are making the value the mean/median/mode of the respective column or deleting the column all together. Deleting the columns would not necessarily be bad given that there are only 99/26651 with NA data. This would mean we are dropping 0.37% of the data.

Using the mean of the columns is also not necessarily bad but can be refined. One way to do this would be to, for example, find, for each specific home team, the average pts scored.


From this table it appears that all NaN values are in the same rows. To double check, we'll check the sums of NaN values again.

Finding the team names for which the data is missing.

In [8]:
teams = pd.read_csv('/kaggle/input/nba-games/teams.csv')
teams.shape

(30, 14)

In [9]:
teams.isna().sum()

LEAGUE_ID             0
TEAM_ID               0
MIN_YEAR              0
MAX_YEAR              0
ABBREVIATION          0
NICKNAME              0
YEARFOUNDED           0
CITY                  0
ARENA                 0
ARENACAPACITY         4
OWNER                 0
GENERALMANAGER        0
HEADCOACH             0
DLEAGUEAFFILIATION    0
dtype: int64

We'll rename the "TEAM_ID" column in teams to "HOME_TEAM_ID" so that the dataframes can be merged

In [10]:
teams = teams.rename(columns = {"TEAM_ID":"HOME_TEAM_ID"})
teams.columns

Index(['LEAGUE_ID', 'HOME_TEAM_ID', 'MIN_YEAR', 'MAX_YEAR', 'ABBREVIATION',
       'NICKNAME', 'YEARFOUNDED', 'CITY', 'ARENA', 'ARENACAPACITY', 'OWNER',
       'GENERALMANAGER', 'HEADCOACH', 'DLEAGUEAFFILIATION'],
      dtype='object')

In [11]:
games_teams_merged = pd.merge(games, teams,on="HOME_TEAM_ID")
games_teams_merged.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,2022-12-22,22200477,Final,1610612740,1610612759,2022,1610612740,126.0,0.484,0.926,...,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
1,2022-12-19,22200457,Final,1610612740,1610612749,2022,1610612740,119.0,0.424,0.708,...,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
2,2022-12-11,22200395,Final,1610612740,1610612756,2022,1610612740,129.0,0.581,0.750,...,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,2022-12-09,22200384,Final,1610612740,1610612756,2022,1610612740,128.0,0.511,0.800,...,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
4,2022-12-07,22200371,Final,1610612740,1610612765,2022,1610612740,104.0,0.459,0.783,...,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate


In [12]:
df2 = games_teams_merged.loc[games_teams_merged.PTS_home.isna()]
display(df2)

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
614,2003-10-18,10300079,Final,1610612740,1610612764,2003,1610612740,NaN,NaN,NaN,...,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
615,2003-10-16,10300059,Final,1610612740,1610612741,2003,1610612740,NaN,NaN,NaN,...,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
1481,2003-10-17,10300065,Final,1610612762,1610612755,2003,1610612762,NaN,NaN,NaN,...,UTA,Jazz,1974,Utah,Vivint Smart Home Arena,20148.0,Greg Miller,Dennis Lindsey,Quin Snyder,Salt Lake City Stars
1482,2003-10-16,10300061,Final,1610612762,1610612753,2003,1610612762,NaN,NaN,NaN,...,UTA,Jazz,1974,Utah,Vivint Smart Home Arena,20148.0,Greg Miller,Dennis Lindsey,Quin Snyder,Salt Lake City Stars
1483,2003-10-14,10300052,Final,1610612762,1610612752,2003,1610612762,NaN,NaN,NaN,...,UTA,Jazz,1974,Utah,Vivint Smart Home Arena,20148.0,Greg Miller,Dennis Lindsey,Quin Snyder,Salt Lake City Stars
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25496,2003-10-14,10300053,Final,1610612757,1610612760,2003,1610612757,NaN,NaN,NaN,...,POR,Trail Blazers,1970,Portland,Moda Center,19980.0,Paul Allen,Neil Olshey,Terry Stotts,No Affiliate
25497,2003-10-09,10300022,Final,1610612757,1610612758,2003,1610612757,NaN,NaN,NaN,...,POR,Trail Blazers,1970,Portland,Moda Center,19980.0,Paul Allen,Neil Olshey,Terry Stotts,No Affiliate
26359,2003-10-19,10300084,Final,1610612744,1610612746,2003,1610612744,NaN,NaN,NaN,...,GSW,Warriors,1946,Golden State,Chase Center,19596.0,Joe Lacob,Bob Myers,Steve Kerr,Santa Cruz Warriors
26360,2003-10-15,10300056,Final,1610612744,1610612760,2003,1610612744,NaN,NaN,NaN,...,GSW,Warriors,1946,Golden State,Chase Center,19596.0,Joe Lacob,Bob Myers,Steve Kerr,Santa Cruz Warriors


In [13]:
df2.isna().sum()

GAME_DATE_EST          0
GAME_ID                0
GAME_STATUS_TEXT       0
HOME_TEAM_ID           0
VISITOR_TEAM_ID        0
SEASON                 0
TEAM_ID_home           0
PTS_home              99
FG_PCT_home           99
FT_PCT_home           99
FG3_PCT_home          99
AST_home              99
REB_home              99
TEAM_ID_away           0
PTS_away              99
FG_PCT_away           99
FT_PCT_away           99
FG3_PCT_away          99
AST_away              99
REB_away              99
HOME_TEAM_WINS         0
LEAGUE_ID              0
MIN_YEAR               0
MAX_YEAR               0
ABBREVIATION           0
NICKNAME               0
YEARFOUNDED            0
CITY                   0
ARENA                  0
ARENACAPACITY         11
OWNER                  0
GENERALMANAGER         0
HEADCOACH              0
DLEAGUEAFFILIATION     0
dtype: int64

In [14]:
#unique_ids = df2["HOME_TEAM_ID"].unique()
_ = df2["ABBREVIATION"].value_counts()
print(_)

LAL    8
SAS    5
DET    5
LAC    5
MIN    4
UTA    4
CHI    4
HOU    4
NYK    4
BKN    4
ORL    4
POR    4
TOR    3
MIL    3
DAL    3
MEM    3
MIA    3
GSW    3
IND    3
SAC    3
BOS    3
PHI    3
DEN    2
PHX    2
OKC    2
ATL    2
WAS    2
CLE    2
NOP    2
Name: ABBREVIATION, dtype: int64


There are 29 of 30 teams have missing values, the top being the lakers with 8. The only team without missing values is the Charlotte Hornets. 

Given the low numbers of missing values per team, the 99 rows with NaN values will be dropped.

In [15]:
games_clean = games_teams_merged.dropna(axis=0)
games_clean.isna().sum()

GAME_DATE_EST         0
GAME_ID               0
GAME_STATUS_TEXT      0
HOME_TEAM_ID          0
VISITOR_TEAM_ID       0
SEASON                0
TEAM_ID_home          0
PTS_home              0
FG_PCT_home           0
FT_PCT_home           0
FG3_PCT_home          0
AST_home              0
REB_home              0
TEAM_ID_away          0
PTS_away              0
FG_PCT_away           0
FT_PCT_away           0
FG3_PCT_away          0
AST_away              0
REB_away              0
HOME_TEAM_WINS        0
LEAGUE_ID             0
MIN_YEAR              0
MAX_YEAR              0
ABBREVIATION          0
NICKNAME              0
YEARFOUNDED           0
CITY                  0
ARENA                 0
ARENACAPACITY         0
OWNER                 0
GENERALMANAGER        0
HEADCOACH             0
DLEAGUEAFFILIATION    0
dtype: int64

Getting the average of home and away scores, all fg%s, ast and reb for each team

In [16]:
unique_t = games_clean.ABBREVIATION.unique()
cols=["PTS_home","FG_PCT_home","FT_PCT_home","FG3_PCT_home","AST_home", "REB_home",
         "PTS_away","FG_PCT_away","FT_PCT_away","FG3_PCT_away","AST_away", "REB_away"]
cols2= cols.insert(0,"ABBREVIATION")
team_means = pd.DataFrame(columns = cols2)

i=0
for t in unique_t:
    t_games = games_clean.loc[games_clean["ABBREVIATION"] == t]
    means = t_games[cols].mean().to_frame().transpose()
    means["ABBREVIATION"] = t
    team_means = team_means.append(means)
    i+=1
    #HOME_TEAM_ID

display(team_means)

/tmp/ipykernel_20/830926374.py:10: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  means = t_games[cols].mean().to_frame().transpose()
/tmp/ipykernel_20/830926374.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  team_means = team_means.append(means)
/tmp/ipykernel_20/830926374.py:10: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  means = t_games[cols].mean().to_frame().transpose()
/tmp/ipykernel_20/830926374.py:12: FutureWarning: The frame.ap

,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,ABBREVIATION
0,103.385057,0.470034,0.752774,0.360905,22.556322,43.637931,98.005747,0.444785,0.761690,0.354489,18.758621,39.359770,UTA
0,101.893362,0.457421,0.745542,0.358482,22.894450,43.758433,99.287269,0.449975,0.756177,0.356473,22.767138,40.848749,CLE
0,102.835414,0.459980,0.767011,0.351469,23.774120,42.632236,101.295119,0.453045,0.761622,0.350910,22.363224,42.492622,ATL
0,102.643083,0.463992,0.777646,0.356264,23.697994,42.473073,98.100317,0.440027,0.755522,0.333277,20.776135,41.652587,BOS
0,102.007177,0.451971,0.765228,0.350065,20.477273,43.279904,102.391148,0.456697,0.757018,0.355895,20.476077,42.360048,NYK
0,104.729911,0.452238,0.748867,0.350980,22.094866,44.056920,100.308036,0.445132,0.747233,0.342571,21.333705,42.712054,HOU
0,102.463820,0.453962,0.766892,0.342883,22.976275,43.287070,102.526690,0.457005,0.758580,0.354918,22.381969,42.398577,MIN
0,105.258137,0.463669,0.774749,0.356364,21.534231,44.122334,101.932660,0.450149,0.761888,0.353254,21.176207,41.132435,OKC
0,105.307049,0.464099,0.765358,0.360559,22.033453,42.364397,105.203106,0.463256,0.767470,0.357591,21.989247,42.602151,SAC
0,103.840757,0.465658,0.750304,0.349842,23.758352,43.145880,100.863029,0.446419,0.754406,0.340589,22.604677,42.233853,LAC


Visualising which team scores and concedes the most points on average when home.

In [17]:
fig = px.bar(team_means,x='ABBREVIATION',y="PTS_home")
fig.show()

In [18]:
fig = px.bar(team_means,x='ABBREVIATION',y='PTS_away')
fig.show()

Are points scored consistent accross the year?

Will also need to repeat the above work to get the averages for when the team is AWAY.

#### Games Details

In [19]:
games_details = pd.read_csv('/kaggle/input/nba-games/games_details.csv')

/tmp/ipykernel_20/2644165350.py:1: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



#### Players

In [20]:
players = pd.read_csv('/kaggle/input/nba-games/players.csv')

#### Ranking

In [21]:
ranking = pd.read_csv('/kaggle/input/nba-games/ranking.csv')

#### Teams